# Template matching of fluorescence data

In [ ]:
from pathlib import Path
import cv2
import numpy as np
import tissue_detection
import matplotlib.pyplot as plt
import mps

Let us download some fluorescence data that was used during the Simber workshop

In [ ]:
tpe_data_path = Path("tpe.npy")

def download_data():
    print("Downloading data. Please wait...")
    link = "https://www.dropbox.com/s/5yyx286v17va5o9/tpe.npy?dl=1"
    import urllib.request
    import time

    urllib.request.urlretrieve(link, tpe_data_path)
    time.sleep(1.0)
    print("Done downloading data")
    
if not tpe_data_path.is_file():
    download_data()
    
data = mps.MPS(tpe_data_path)

Let us extract the first frame and plot it

In [ ]:
img = data.frames[:, :, 0]
plt.imshow(img)

Now lets select the template and also include the pilars in the template

In [ ]:
template = tissue_detection.tpe.TPE1(include_pilars=True)

First let's try to run this using the default settings.

In [ ]:
result = template.match(img)

fig, ax = plt.subplots(2, 3, figsize=(12, 10))
ax[0, 0].imshow(img)
ax[0, 0].imshow(result.result, alpha=0.4)
ax[0, 0].set_title("Match")

ax[0, 1].imshow(result.match_result)
ax[0, 1].set_title("Template match error")
ax[0, 2].imshow(img)
ax[0, 2].set_title("Original image")

ax[1, 0].imshow(result.template)
ax[1, 0].set_title("Template")
ax[1, 1].imshow(result.template_mask)
ax[1, 1].set_title("Template mask")
ax[1, 2].imshow(result.result)
ax[1, 2].set_title("Mask")

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
for i, axi in enumerate(ax.flatten(), start=1):
    mask = np.zeros_like(result.result)
    mask[result.result == i] = 1
    axi.imshow(mask)
    axi.set_title(f"Tissue {i}")

plt.show()

What we see is that the template is way too big. This is because this data is actually downsampled by a factor of 0.5. We can add this information to the matching algorithm

In [ ]:
result = template.match(img, padding=50, scale=0.5)

fig, ax = plt.subplots(2, 3, figsize=(12, 10))
ax[0, 0].imshow(img)
ax[0, 0].imshow(result.result, alpha=0.4)
ax[0, 0].set_title("Match")

ax[0, 1].imshow(result.match_result)
ax[0, 1].set_title("Template match error")
ax[0, 2].imshow(img)
ax[0, 2].set_title("Original image")

ax[1, 0].imshow(result.template)
ax[1, 0].set_title("Template")
ax[1, 1].imshow(result.template_mask)
ax[1, 1].set_title("Template mask")
ax[1, 2].imshow(result.result)
ax[1, 2].set_title("Mask")

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
for i, axi in enumerate(ax.flatten(), start=1):
    mask = np.zeros_like(result.result)
    mask[result.result == i] = 1
    axi.imshow(mask)
    axi.set_title(f"Tissue {i}")

plt.show()

This looks better but it is still not a good match. We can try to invert the pixel values in the image

In [ ]:
result = template.match(img, padding=50, scale=0.5, invert=True)

fig, ax = plt.subplots(2, 3, figsize=(12, 10))
ax[0, 0].imshow(img)
ax[0, 0].imshow(result.result, alpha=0.4)
ax[0, 0].set_title("Match")

ax[0, 1].imshow(result.match_result)
ax[0, 1].set_title("Template match error")
ax[0, 2].imshow(img)
ax[0, 2].set_title("Original image")

ax[1, 0].imshow(result.template)
ax[1, 0].set_title("Template")
ax[1, 1].imshow(result.template_mask)
ax[1, 1].set_title("Template mask")
ax[1, 2].imshow(result.result)
ax[1, 2].set_title("Mask")

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
for i, axi in enumerate(ax.flatten(), start=1):
    mask = np.zeros_like(result.result)
    mask[result.result == i] = 1
    axi.imshow(mask)
    axi.set_title(f"Tissue {i}")

plt.show()

Still, the results doesn't look great. Now let us try to help the algorithm by removing noise from the image. We can do this by first blurring the image, and then perform a thresholding

In [ ]:

blur = cv2.GaussianBlur(img, (25, 25), 10)
th_value, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


fig, ax = plt.subplots(1, 2, figsize=(10, 6))
ax[0].imshow(img)
ax[0].imshow(thresh, alpha=0.1, cmap="gray")
ax[1].imshow(thresh, cmap="gray")
plt.show()

We can now use this thresholded image as input to the algorithm instead

In [ ]:


result = template.match(thresh, padding=50, scale=0.5)

fig, ax = plt.subplots(2, 3, figsize=(12, 10))
ax[0, 0].imshow(img)
ax[0, 0].imshow(result.result, alpha=0.4)
ax[0, 0].set_title("Match")

ax[0, 1].imshow(result.match_result)
ax[0, 1].set_title("Template match error")
ax[0, 2].imshow(img)
ax[0, 2].set_title("Original image")

ax[1, 0].imshow(result.template)
ax[1, 0].set_title("Template")
ax[1, 1].imshow(result.template_mask)
ax[1, 1].set_title("Template mask")
ax[1, 2].imshow(result.result)
ax[1, 2].set_title("Mask")

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
for i, axi in enumerate(ax.flatten(), start=1):
    mask = np.zeros_like(result.result)
    mask[result.result == i] = 1
    axi.imshow(mask)
    axi.set_title(f"Tissue {i}")

plt.show()

And we see that we now got a good match. We can now use this mask to compute the signal from the four tissues.

In [ ]:
fig, ax = plt.subplots()
for i in range(1, 5):
    mask = np.zeros(result.result.shape, dtype=bool)
    mask[result.result == i] = True
    y = data.frames[mask].mean(axis=0)
    ax.plot(data.time_stamps, y, label=f"tissue {i}")

ax.legend()
plt.show()